# Group 08: Object detection for Images on Embedded Systems
### The network based on YOLOv2-tiny
###  Zhu Shien(G1801969B) & Mo Xiaoyun(G1702922E)

  This is the main notebook using the newest YOLO-like edition of Network to illustrate the development process of our work.<br>
  The data preprocessing is in another notebook. <br>
  There are three stages in this notebook: <br>
  * training, 
  * inference,
  * comparison between different networks we developed.
## Stage one: training
Content:<br>
*    import libries, 
*    functions for calculating IOU, 
*    function for getting classification accuracy, 
*    network configurations, 
*    training the network
*    the result analysis

### Import the libraries and our codes

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import utils
import os
import time
# These are our codes,
# readin_v5 is the latest version of reading in images and XML,
# mynet contains all the Neural Networks we created
import readin_v5
import mynet

### A function for calculating the IOU of two bound boxes

In [2]:
# input the xmin, ymin, w, h of two boxes, then return the IOU
def cal_iou(infer,target):
    x=infer[0]
    y=infer[1]
    w=infer[2]
    h=infer[3]
    xt=target[0]
    yt=target[1]
    wt=target[2]
    ht=target[3]
    # case 0: when contained by the target
    if ((x+w)>=xt) and ((y+h)>=yt) and ((x+w)<=xt+wt) and ((y+h)<=yt+ht) and (x>=xt) and (y>=yt) and (x<=xt+wt) and (y<=yt+ht):
        same=w*h
        iou=same/(wt*ht)
    # case 1: when right upper part of infer is the same with target
    elif ((x+w)>=xt) and ((y+h)>=yt) and ((x+w)<=xt+wt) and ((y+h)<=yt+ht):
        same=(x+w-xt)*(y+h-yt)
        iou=same/(w*h+wt*ht-same)
    # case 2: right down part
    elif ((x+w)>=xt) and (y>=yt) and ((x+w)<=xt+wt) and (y<=yt+ht):
        same=(x+w-xt)*(yt+ht-y)
        iou=same/(w*h+wt*ht-same)
    # case 3: left upper part
    elif (x>=xt) and ((y+h)>=yt) and (x<=xt+wt) and ((y+h)<=yt+ht):
        same=(xt+wt-x)*(y+h-yt)
        iou=same/(w*h+wt*ht-same)
    # case 4: left down part
    elif (x>=xt) and (y>=yt) and (x<=xt+wt) and (y<=yt+ht):
        same=(xt+wt-x)*(yt+ht-y)
        iou=same/(w*h+wt*ht-same)
    # case 5: contain the target
    elif ((xt+wt)>=x) and ((yt+ht)>=y) and ((xt+wt)<=x+w) and ((yt+ht)<=y+h) and (xt>=x) and (yt>=y) and (xt<=x+w) and (yt<=y+h):
        same=wt*ht
        iou=same/(w*h)
    else :
        same=0.0
        iou=0.0
    del x,y,w,h,xt,yt,wt,ht,same
    return iou

### Testing code for the cal_iou in different cases

In [3]:
target=[1.0,1.0,2.0,2.0]
# none
infer=[-1.0,-1.0,0.0,0.0]
print(cal_iou(infer,target))
# right upper part of infer is the same with target
infer=[-1.0,-1.0,3.0,3.0]
print(cal_iou(infer,target))
# right down
infer=[-1.0,2.0,3.0,3.0]
print(cal_iou(infer,target))
# left upper
infer=[2.0,2.0,3.0,3.0]
print(cal_iou(infer,target))
# left down
infer=[2.0,-1.0,3.0,3.0]
print(cal_iou(infer,target))
# contained by the target
infer=[2.0,2.0,1.0,1.0]
print(cal_iou(infer,target))
# contain the target
infer=[-1.0,-1.0,5.0,5.0]
print(cal_iou(infer,target))
# same
print(cal_iou(target,target))

0.0
0.08333333333333333
0.08333333333333333
0.08333333333333333
0.08333333333333333
0.25
0.16
1.0


### Calculate the mean IOU for a batch<br>
Only pass the x,y,w,h into the cal_iou function<br>
The outputs are a little bit large, so we devid them by 10 or 100, thus 30.5 -> 3.05, 640 -> 6.40<br>
The ratio here is the same with the network design.<br>
The output of x,y,w,h are all 10% of the real x,y,w,h, and both are devided by 10,<br>
thus the numbers are 10 and 100.

In [4]:
def get_iou(out_prob,real_label,bs):
    iou_count=0.0
    for i in range(bs):
        iou_count=iou_count+cal_iou(out_prob[i,80:84]/10,real_label[i,1:5].float()/100)
    return(iou_count/bs)

### A function to get the classification accuracy
This function records the location of the highest value in a vector and campare with the real label, return the number of accurate labels in a batch.

In [5]:
def get_accu(out_prob,real_label,bs):
    count=0
    for i in range(bs):
        temp=0.0
        maxp=0
        for j in range(80):
            if temp<out_prob[i,j]:
                temp=out_prob[i,j]
                maxp=j
        if maxp==real_label[i,0]:
            count=count+1
    return(count)

### Code for training
Set the data path<br>
This is the directionary organization:<br>
codes.ipynb<br>
training_record.txt<br>
network_weight.pt<br>
--train data<br>
--train label<br>
--small data<br>
--small label<br>
--test data<br>
--test label

In [6]:
mydir=os.getcwd()
train_data_dir=mydir+"\\train_data"
train_label_dir=mydir+"\\train_label"
small_data_dir=mydir+"\\small_data"
small_label_dir=mydir+"\\small_label"

### Configure the network and training process

In [7]:
# Define a new neural network,
# the original name is for identifying the training stragedy and network configurationsn    
net = mynet.yololike_v2()
origname="yololike_v2_6000"
record=open(origname+"_record.txt",mode='a')

# There are 6000 images for training and 900 images for small scale test
bs=20
lr=0.1   # the original learning rate is 0.1, and it will decrease when training
epoch_num=4   
train_data_num=100  # this number is for reading in images in parts, we cannot load in all images in RAM
train_total=6000  
small_num=900  # this is for small scale test within the training stage
iter_num=int(train_data_num/bs)  # this is for iteration when training 
criterion1 =nn.CrossEntropyLoss()  # there are two criteria for the loss function
criterion2 =nn.MSELoss()

### Start training, and record the whole process

In [8]:
start_time = time.time()  
for epoch in range(epoch_num):
    # show and update learning rate
    print("lr=",lr)
    record.write("\nLearning rate: "+'{0:.6f}'.format(lr))
    for step in range(int(train_total/train_data_num)):
        # update the learning rate
        if (step>0) and (step%5==0) :
            del optimizer
            if lr>0.002 : # so that the lr is not too small
                lr=lr/2
                print("lr=",lr)
                record.write("\nLearning rate: "+'{0:.6f}'.format(lr))
        optimizer=torch.optim.SGD(net.parameters(),lr)
        
        # read in the data
        train_data = readin_v5.read_part_image(train_data_dir,step*train_data_num,train_data_num)
        train_label= readin_v5.read_part_xml(train_label_dir,step*train_data_num,train_data_num)
        print(train_data.size(),train_data.type())
        
        # train with batches
        accu_count=0
        step_loss=0.0
        iou_count=0.0
        for iter in range(iter_num):
            # delete the variables of last iteration, though not necessary
            if iter>0 :
                del indices
                del minibatch_data
                del minibatch_label
                del inputs
                del prob
                del loss
                del accu_batch
                
            # create a minibatch
            indices=iter*bs
            minibatch_data = train_data[indices:indices+bs]
            minibatch_label= train_label[indices:indices+bs]
    
            # feed the input to the net
            inputs=minibatch_data
            inputs.requires_grad_()
            prob=net(inputs)
    
            # update the weights
            loss=criterion1(prob[:,0:80], minibatch_label[:,0])+criterion2(prob[:,80:84]/10, minibatch_label[:,1:5].float()/100)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            step_loss=step_loss+loss.item()
            
            # get the accuracy and IOU
            accu_batch=get_accu(prob,minibatch_label,bs)
            accu_count=accu_count+accu_batch
            iou_batch=get_iou(prob,minibatch_label,bs)
            iou_item=iou_batch
            del iou_batch # sometimes somehow it should be deleted here
            iou_count=iou_count+iou_item
            
            # Show and record the results to get information about the training process
            print("batch no."+str(step*iter_num+iter)+" loss: "+'{0:.4f}'.format(loss.item())+
                         " time: "+'{0:.2f}'.format(time.time()-start_time)+
                         " accu="+str(accu_batch)+" iou="+'{0:.4f}'.format(iou_item))
            record.write("\nbatch no."+str(step*iter_num+iter)+" loss: "+'{0:.4f}'.format(loss.item())+
                         " time: "+'{0:.2f}'.format(time.time()-start_time)+
                         " accu="+str(accu_batch)+" iou="+'{0:.4f}'.format(iou_item))
        print("step loss: ",'{0:.2f}'.format(step_loss/iter_num)," accu=",'{0:.4f}'.format(accu_count/train_data_num),
              " iou="+'{0:.4f}'.format(iou_count/iter_num))
        record.write("\nstep loss: "+'{0:.4f}'.format(step_loss/iter_num)+" accu="+'{0:.4f}'.format(accu_count/train_data_num)+
                     " iou="+'{0:.4f}'.format(iou_count/iter_num)+" time: "+'{0:.2f}'.format(time.time()-start_time))   
    del train_data
    del train_label
    del indices
    del minibatch_data
    del minibatch_label
    del inputs
    del prob
    del loss
    
    # save the network in every epoch
    net_name=origname+"_epoch_"+str(epoch)+".pt"
    torch.save(net.state_dict(),net_name)
    
    # small scale test
    small_data = readin_v5.read_part_image(small_data_dir,0,small_num)
    small_label= readin_v5.read_part_xml(small_label_dir,0,small_num)
    small_loss=0.0
    accu_count=0
    iou_count=0.0
    for iter in range(int(small_num/bs)):
        if iter>0 :
            del indices
            del minibatch_data
            del minibatch_label
            del inputs
            del prob
            del loss
            del accu_batch
            del iou_item
                
        # create a minibatch
        indices=iter*bs
        minibatch_data = small_data[indices: indices+bs]
        minibatch_label= small_label[indices: indices+bs]
    
        # feed the input to the net
        inputs=minibatch_data
        prob=net(inputs)
        
        # get the accuracy and IOU
        accu_batch=get_accu(prob,minibatch_label,bs)
        accu_count=accu_count+accu_batch
        # the iou calculation is somehow strange, I have to delete the variable immediately,
        # otherwise the RAM will be fullly used, BUT I returned the iou as float, actually it is a tensor
        iou_batch=get_iou(prob,minibatch_label,bs)
        iou_item=iou_batch.item()
        del iou_batch 
        iou_count=iou_count+iou_item
        
        # get the loss
        loss=criterion1(prob[:,0:80], minibatch_label[:,0])+criterion2(prob[:,80:84]/10, minibatch_label[:,1:5].float()/100)
        small_loss=small_loss+loss.item()
        
        # show and record
        print("small_batch no."+str(iter)+" loss: "+'{0:.4f}'.format(loss.item())+
                         " time: "+'{0:.2f}'.format(time.time()-start_time)+
                         " accu="+str(accu_batch)+" iou="+'{0:.4f}'.format(iou_item))
        record.write("\nsmall_batch no."+str(step*iter_num+iter)+" loss: "+'{0:.4f}'.format(loss.item())+
                         " time: "+'{0:.2f}'.format(time.time()-start_time)+
                     " accu="+str(accu_batch)+" iou="+'{0:.4f}'.format(iou_item))
    del accu_batch
    del iou_item
    del indices
    del minibatch_data
    del minibatch_label
    del inputs
    del prob
    del loss   
    del small_data
    del small_label
    print("small loss:",'{0:.2f}'.format(small_loss/(small_num/bs))," accu=",'{0:.4f}'.format(accu_count/small_num),
          " iou=",'{0:.4f}'.format(iou_count/(small_num/bs)))
    record.write("small loss:"+'{0:.4f}'.format(small_loss/(small_num/bs))+" accuracy:"+'{0:.4f}'.format(accu_count/small_num)+
                 " iou="+'{0:.4f}'.format(iou_count/(small_num/bs)))
record.close()


lr= 0.1
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.0 loss: 7.4080 time: 6.56 accu=0 iou=0.0000
batch no.1 loss: 6.8623 time: 11.97 accu=0 iou=0.0000
batch no.2 loss: 6.4427 time: 17.55 accu=0 iou=0.0000
batch no.3 loss: 7.8800 time: 23.03 accu=2 iou=0.0000
batch no.4 loss: 6.2113 time: 28.52 accu=0 iou=0.0000
step loss:  6.96  accu= 0.0200  iou=0.0000
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.5 loss: 8.7728 time: 34.85 accu=0 iou=0.0000
batch no.6 loss: 5.2598 time: 40.31 accu=3 iou=0.0000
batch no.7 loss: 5.9408 time: 45.74 accu=3 iou=0.0006
batch no.8 loss: 6.7315 time: 51.34 accu=2 iou=0.0014
batch no.9 loss: 7.1815 time: 56.62 accu=1 iou=0.0000
step loss:  6.78  accu= 0.0900  iou=0.0004
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.10 loss: 7.8383 time: 64.23 accu=2 iou=0.0000
batch no.11 loss: 5.1519 time: 69.83 accu=0 iou=0.0000
batch no.12 loss: 4.8363 time: 75.56 accu=2 iou=0.0178
batch no.13 loss: 4.2703 time: 81.15 accu=3 iou=0.0181


torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.110 loss: 0.7078 time: 649.89 accu=17 iou=0.0406
batch no.111 loss: 0.7133 time: 655.25 accu=19 iou=0.1448
batch no.112 loss: 0.4962 time: 660.60 accu=19 iou=0.1373
batch no.113 loss: 0.7110 time: 665.99 accu=19 iou=0.0872
batch no.114 loss: 0.8274 time: 671.42 accu=17 iou=0.0480
step loss:  0.69  accu= 0.9100  iou=0.0916
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.115 loss: 0.8633 time: 678.97 accu=19 iou=0.1332
batch no.116 loss: 1.1482 time: 684.38 accu=13 iou=0.0632
batch no.117 loss: 0.9205 time: 689.86 accu=16 iou=0.0790
batch no.118 loss: 0.5864 time: 695.27 accu=18 iou=0.1232
batch no.119 loss: 0.8896 time: 700.73 accu=16 iou=0.0970
step loss:  0.88  accu= 0.8200  iou=0.0991
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.120 loss: 0.8771 time: 718.67 accu=17 iou=0.0969
batch no.121 loss: 1.1218 time: 724.06 accu=17 iou=0.0546
batch no.122 loss: 0.7830 time: 729.44 accu=17 iou=0.0992
batch no.123 

batch no.216 loss: 0.9648 time: 1284.24 accu=17 iou=0.1029
batch no.217 loss: 0.6503 time: 1289.82 accu=19 iou=0.0431
batch no.218 loss: 0.6691 time: 1295.29 accu=18 iou=0.0453
batch no.219 loss: 0.5536 time: 1300.68 accu=18 iou=0.0414
step loss:  0.72  accu= 0.9000  iou=0.0646
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.220 loss: 0.6815 time: 1308.18 accu=20 iou=0.0493
batch no.221 loss: 0.6709 time: 1313.71 accu=19 iou=0.0770
batch no.222 loss: 0.7124 time: 1319.11 accu=17 iou=0.0759
batch no.223 loss: 0.6789 time: 1324.43 accu=19 iou=0.0925
batch no.224 loss: 0.7842 time: 1329.92 accu=17 iou=0.1249
step loss:  0.71  accu= 0.9200  iou=0.0839
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.225 loss: 0.3884 time: 1337.64 accu=18 iou=0.0502
batch no.226 loss: 0.8054 time: 1342.95 accu=17 iou=0.1080
batch no.227 loss: 0.5641 time: 1348.22 accu=20 iou=0.0895
batch no.228 loss: 0.7129 time: 1353.58 accu=19 iou=0.0563
batch no.229 loss: 0.7183 time: 1358.92 accu=18 i

small_batch no.28 loss: 0.7601 time: 1841.15 accu=17 iou=0.1426
small_batch no.29 loss: 0.5699 time: 1843.54 accu=19 iou=0.0679
small_batch no.30 loss: 0.5668 time: 1845.86 accu=20 iou=0.0844
small_batch no.31 loss: 0.7631 time: 1848.29 accu=18 iou=0.0942
small_batch no.32 loss: 0.7893 time: 1850.68 accu=18 iou=0.1244
small_batch no.33 loss: 0.5273 time: 1853.06 accu=18 iou=0.1482
small_batch no.34 loss: 0.6418 time: 1855.43 accu=18 iou=0.0807
small_batch no.35 loss: 0.4885 time: 1857.89 accu=19 iou=0.1023
small_batch no.36 loss: 0.6912 time: 1860.31 accu=17 iou=0.0234
small_batch no.37 loss: 0.6238 time: 1862.81 accu=19 iou=0.2515
small_batch no.38 loss: 0.6628 time: 1865.20 accu=19 iou=0.0399
small_batch no.39 loss: 0.5707 time: 1867.61 accu=18 iou=0.0732
small_batch no.40 loss: 0.5789 time: 1869.96 accu=19 iou=0.0801
small_batch no.41 loss: 0.5634 time: 1872.37 accu=20 iou=0.1623
small_batch no.42 loss: 0.5662 time: 1874.78 accu=19 iou=0.1324
small_batch no.43 loss: 0.5722 time: 187

batch no.93 loss: 0.7008 time: 2405.58 accu=18 iou=0.0931
batch no.94 loss: 0.7969 time: 2411.05 accu=17 iou=0.1175
step loss:  0.62  accu= 0.9200  iou=0.1008
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.95 loss: 0.5808 time: 2417.36 accu=18 iou=0.0648
batch no.96 loss: 0.7753 time: 2422.61 accu=18 iou=0.0460
batch no.97 loss: 0.3315 time: 2428.05 accu=20 iou=0.1073
batch no.98 loss: 0.4389 time: 2433.58 accu=19 iou=0.1720
batch no.99 loss: 0.8215 time: 2438.92 accu=16 iou=0.1120
step loss:  0.59  accu= 0.9100  iou=0.1004
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.100 loss: 0.4355 time: 2445.08 accu=20 iou=0.1334
batch no.101 loss: 0.5388 time: 2450.49 accu=19 iou=0.1261
batch no.102 loss: 0.5862 time: 2455.81 accu=18 iou=0.0830
batch no.103 loss: 0.3926 time: 2461.22 accu=20 iou=0.1379
batch no.104 loss: 0.3286 time: 2466.61 accu=20 iou=0.0973
step loss:  0.46  accu= 0.9700  iou=0.1155
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.105 loss: 0.81

...

batch no.182 loss: 0.2745 time: 4687.94 accu=20 iou=0.0676
batch no.183 loss: 0.5026 time: 4693.38 accu=20 iou=0.1102
batch no.184 loss: 0.7741 time: 4698.83 accu=16 iou=0.0625
step loss:  0.51  accu= 0.9300  iou=0.0803
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.185 loss: 0.3556 time: 4704.94 accu=19 iou=0.0725
batch no.186 loss: 0.4265 time: 4710.34 accu=18 iou=0.0931
batch no.187 loss: 0.4841 time: 4715.80 accu=19 iou=0.1085
batch no.188 loss: 0.3455 time: 4721.30 accu=19 iou=0.0500
batch no.189 loss: 1.0460 time: 4726.67 accu=16 iou=0.0471
step loss:  0.53  accu= 0.9100  iou=0.0743
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.190 loss: 0.4560 time: 4732.97 accu=19 iou=0.0086
batch no.191 loss: 0.3728 time: 4738.33 accu=20 iou=0.0927
batch no.192 loss: 0.3692 time: 4743.70 accu=20 iou=0.1631
batch no.193 loss: 0.5660 time: 4749.06 accu=18 iou=0.1085
batch no.194 loss: 0.3330 time: 4754.50 accu=19 iou=0.0969
step loss:  0.42  accu= 0.9600  iou=0.0940
torch.

batch no.289 loss: 0.4544 time: 5283.86 accu=19 iou=0.1153
step loss:  0.42  accu= 0.9500  iou=0.0703
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.290 loss: 0.6169 time: 5290.11 accu=17 iou=0.0728
batch no.291 loss: 0.4770 time: 5295.47 accu=19 iou=0.0705
batch no.292 loss: 0.2389 time: 5300.88 accu=20 iou=0.0939
batch no.293 loss: 0.9063 time: 5306.30 accu=16 iou=0.0229
batch no.294 loss: 0.2308 time: 5311.64 accu=20 iou=0.1479
step loss:  0.49  accu= 0.9200  iou=0.0816
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.295 loss: 0.5216 time: 5317.74 accu=20 iou=0.1165
batch no.296 loss: 0.4647 time: 5323.16 accu=18 iou=0.1348
batch no.297 loss: 0.3008 time: 5328.53 accu=20 iou=0.0677
batch no.298 loss: 0.4261 time: 5333.97 accu=19 iou=0.1665
batch no.299 loss: 0.2725 time: 5339.39 accu=20 iou=0.0320
step loss:  0.40  accu= 0.9700  iou=0.1035
small_batch no.0 loss: 0.5729 time: 5347.55 accu=20 iou=0.0837
small_batch no.1 loss: 0.3546 time: 5349.89 accu=20 iou=0.093

batch no.58 loss: 0.3904 time: 5784.53 accu=19 iou=0.0478
batch no.59 loss: 0.2916 time: 5789.94 accu=20 iou=0.0398
step loss:  0.37  accu= 0.9500  iou=0.0970
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.60 loss: 0.4271 time: 5796.16 accu=20 iou=0.0320
batch no.61 loss: 0.4114 time: 5801.56 accu=20 iou=0.0758
batch no.62 loss: 0.3399 time: 5806.87 accu=20 iou=0.0693
batch no.63 loss: 0.3236 time: 5812.36 accu=20 iou=0.1576
batch no.64 loss: 0.3557 time: 5817.69 accu=20 iou=0.0062
step loss:  0.37  accu= 1.0000  iou=0.0682
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.65 loss: 0.3286 time: 5824.00 accu=20 iou=0.1401
batch no.66 loss: 0.2635 time: 5829.37 accu=20 iou=0.0627
batch no.67 loss: 0.4313 time: 5834.72 accu=20 iou=0.1049
batch no.68 loss: 0.2065 time: 5840.06 accu=20 iou=0.1542
batch no.69 loss: 0.3090 time: 5845.45 accu=20 iou=0.1230
step loss:  0.31  accu= 1.0000  iou=0.1170
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.70 loss: 0.4078 tim

torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.165 loss: 0.4049 time: 6380.60 accu=19 iou=0.2031
batch no.166 loss: 0.2477 time: 6386.02 accu=20 iou=0.1465
batch no.167 loss: 0.5254 time: 6391.40 accu=17 iou=0.0980
batch no.168 loss: 0.3422 time: 6396.72 accu=19 iou=0.0774
batch no.169 loss: 0.3508 time: 6402.13 accu=20 iou=0.1225
step loss:  0.37  accu= 0.9500  iou=0.1295
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.170 loss: 0.3409 time: 6408.43 accu=20 iou=0.1023
batch no.171 loss: 0.5241 time: 6413.86 accu=19 iou=0.0789
batch no.172 loss: 0.7199 time: 6419.22 accu=18 iou=0.0639
batch no.173 loss: 0.3986 time: 6424.64 accu=19 iou=0.1851
batch no.174 loss: 0.3075 time: 6430.00 accu=20 iou=0.0828
step loss:  0.46  accu= 0.9600  iou=0.1026
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.175 loss: 0.3855 time: 6436.21 accu=19 iou=0.0863
batch no.176 loss: 0.2255 time: 6441.58 accu=20 iou=0.0381
batch no.177 loss: 0.3324 time: 6446.92 accu=20 iou=0.0664


batch no.271 loss: 0.4159 time: 6971.26 accu=20 iou=0.1267
batch no.272 loss: 0.5258 time: 6976.65 accu=18 iou=0.1310
batch no.273 loss: 0.2624 time: 6982.09 accu=20 iou=0.0733
batch no.274 loss: 0.3187 time: 6987.43 accu=20 iou=0.1672
step loss:  0.35  accu= 0.9800  iou=0.1148
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.275 loss: 0.2832 time: 6993.60 accu=20 iou=0.1675
batch no.276 loss: 0.3921 time: 6998.92 accu=19 iou=0.1230
batch no.277 loss: 0.4179 time: 7004.29 accu=20 iou=0.1147
batch no.278 loss: 0.2238 time: 7009.70 accu=20 iou=0.0954
batch no.279 loss: 0.3931 time: 7015.10 accu=19 iou=0.1205
step loss:  0.34  accu= 0.9800  iou=0.1242
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no.280 loss: 0.2348 time: 7021.29 accu=20 iou=0.0819
batch no.281 loss: 0.3765 time: 7026.63 accu=20 iou=0.1667
batch no.282 loss: 0.2598 time: 7031.96 accu=20 iou=0.0795
batch no.283 loss: 0.3055 time: 7037.44 accu=19 iou=0.1875
batch no.284 loss: 0.6204 time: 7042.91 accu=18 i

### Training process analysis
First, about the learning rate<br>
*    The learning rate is updated every 500 images. This stratery is obtained by practice. The are several steps for the weights to be updated and the loss of training data drops steadily, so it is useful and effective.
*    It is not updated every epoch, because one epoch contains 6000 images and it is quite a long time before updating the lr, which may causes the learning rate is too large to converge and the loss won't drop for a long time.<br>
Second, about the batch size<br>
*    The batch size was fixed to 20 images in the begining and did not cause any problem, so we keep it.<br>
Third, about the epoch number<br>
*    We tried different epoch_num, 2,4,8,10,20, and 2 or 4 epochs is enough in our training strategy, because the learning rate is very small after one epoch and the weights won't change much in the next epoch.
*    Though we only train it for 2 or 4 epochs, the classification accuracy has reached more than 90%, we think it is enough.
Last, about the recording and showing of training process.<br>
*    We print every batch's loss and accuracy to keep an eye on the network accuracy and the effectiveness of our training strategy. The record is very useful when analysing the results and making new decisions.

## The inference stage
Here are the codes for inference the result with the test data<br>
* Load in the network trained before
* Configure the inference process
* Start testing
* Get evaluation result
* Inference Analysis

### Load in the network we trained before
For illustration, we load in as CPU mode and run the code here

In [9]:
net = mynet.yololike_v2()
mydir=os.getcwd()
net.load_state_dict(torch.load(mydir+"\yololike_v2_6000_epoch_3.pt",map_location='cpu'))

### Configure the inference process
We load in the test data in steps, and inference with batch

In [10]:
test_num=2500
step_num=100
bs=20
mydir=os.getcwd()
test_data_dir=mydir+"\\test_data"
test_label_dir=mydir+"\\test_label"
print(test_data_dir,test_label_dir)

C:\Users\Shaun\CE7454_2018\codes\DeepLearning\test_data C:\Users\Shaun\CE7454_2018\codes\DeepLearning\test_label


### Start testing
We count the testing time from reading in the data and only record the result without getting the accuracy<br>
The loading data part consume comparable time as the net.forward()

In [11]:
# record test result
results=torch.zeros([test_num,84],dtype=torch.float)

start_time=time.time()
for step in range(int(test_num/step_num)):
    test_data = readin_v5.read_part_image(test_data_dir,step*step_num,step_num)
    for iter in range(int(step_num/bs)):        
        # create a minibatch
        indices=iter*bs
        prob=net.forward(test_data[indices: indices+bs])
        ## this is the most important part here to avoid RAM over use: only asign prob.data to results
        results[step*step_num+indices: step*step_num+indices+bs,:]=prob.data 
        del indices
        del prob
    del test_data
    print("step_time:",time.time()-start_time)
print("total time:",time.time()-start_time)

step_time: 12.999080181121826
step_time: 25.518290758132935
step_time: 38.03605246543884
step_time: 50.5229856967926
step_time: 62.93101620674133
step_time: 75.43343353271484
step_time: 87.88944888114929
step_time: 100.58496356010437
step_time: 113.16196274757385
step_time: 125.80425024032593
step_time: 138.41537642478943
step_time: 151.10597014427185
step_time: 163.8887221813202
step_time: 177.64114117622375
step_time: 190.42468166351318
step_time: 203.17652869224548
step_time: 215.75562381744385
step_time: 228.46642684936523
step_time: 241.1421046257019
step_time: 253.72224855422974
step_time: 266.600412607193
step_time: 279.258362531662
step_time: 291.85504817962646
step_time: 304.4667844772339
step_time: 316.96875524520874
total time: 316.96875524520874


### Get evaluation result
Modify and reuse the code of training stage ^_^

In [12]:
# record the inference accuracy
origname="yololike_v2_6000"
record=open(origname+"_test.txt",mode='a')

# setup for getting the accuracy
test_loss=0.0
accu_count=0
iou_count=0.0
criterion1 =nn.CrossEntropyLoss()
criterion2 =nn.MSELoss()

start_time=time.time()
test_label= readin_v5.read_part_xml(test_label_dir,0,test_num)
for iter in range(int(test_num/bs)):
        indices=iter*bs
        minibatch_label= test_label[indices: indices+bs]
        prob=results[indices: indices+bs,:]
        
        # get the accuracy and IOU
        accu_batch=get_accu(prob,minibatch_label,bs)
        accu_count=accu_count+accu_batch
        iou_batch=get_iou(prob,minibatch_label,bs)
        iou_item=iou_batch.item()
        del iou_batch
        iou_count=iou_count+iou_item
        
        # get the loss
        loss=criterion1(prob[:,0:80], minibatch_label[:,0])+criterion2(prob[:,80:84]/10, minibatch_label[:,1:5].float()/100)
        test_loss=test_loss+loss.item()
        
        # show and record
        print("test_batch no."+str(iter)+" loss: "+'{0:.4f}'.format(loss.item())+
                         " time: "+'{0:.2f}'.format(time.time()-start_time)+
                         " accu="+str(accu_batch)+" iou="+'{0:.4f}'.format(iou_item))
        record.write("\ntest_batch no."+str(iter)+" loss: "+'{0:.4f}'.format(loss.item())+
                         " time: "+'{0:.2f}'.format(time.time()-start_time)+
                     " accu="+str(accu_batch)+" iou="+'{0:.4f}'.format(iou_item))

print("test loss:",'{0:.2f}'.format(test_loss/(test_num/bs))," accu=",'{0:.4f}'.format(accu_count/test_num),
          " iou=",'{0:.4f}'.format(iou_count/(test_num/bs)))
record.write("\ntest loss:"+'{0:.4f}'.format(test_loss/(test_num/bs))+" accuracy:"+'{0:.4f}'.format(accu_count/test_num)+
                 " iou="+'{0:.4f}'.format(iou_count/(test_num/bs)))
record.close()

test_batch no.0 loss: 0.2930 time: 1.03 accu=19 iou=0.1258
test_batch no.1 loss: 0.5534 time: 1.03 accu=17 iou=0.0365
test_batch no.2 loss: 0.4137 time: 1.05 accu=18 iou=0.1126
test_batch no.3 loss: 0.3627 time: 1.06 accu=20 iou=0.1628
test_batch no.4 loss: 0.3345 time: 1.06 accu=19 iou=0.0865
test_batch no.5 loss: 0.3737 time: 1.08 accu=20 iou=0.1128
test_batch no.6 loss: 0.4868 time: 1.08 accu=19 iou=0.1397
test_batch no.7 loss: 0.2922 time: 1.09 accu=19 iou=0.1951
test_batch no.8 loss: 0.2506 time: 1.09 accu=20 iou=0.0878
test_batch no.9 loss: 0.3827 time: 1.11 accu=20 iou=0.1577
test_batch no.10 loss: 0.3304 time: 1.11 accu=20 iou=0.1101
test_batch no.11 loss: 0.3361 time: 1.12 accu=20 iou=0.1333
test_batch no.12 loss: 0.3954 time: 1.14 accu=20 iou=0.0581
test_batch no.13 loss: 0.2998 time: 1.14 accu=20 iou=0.0409
test_batch no.14 loss: 0.4189 time: 1.16 accu=19 iou=0.0908
test_batch no.15 loss: 0.2685 time: 1.16 accu=19 iou=0.1833
test_batch no.16 loss: 0.5010 time: 1.17 accu=19 i

### Inference process analysis
We mainly take inference time into account and the quicker, the better<br>
We split getting accuracy from inference, but the bottleneck here is not loading in data, for it shows in our another notebook that it only takes less than 1s to read in 100 images, but more than 12s to get the results in CPU mode.<br>
So inference on GPU is a good way to solve it. When we test it on Nvidia Jetson TX2, it only need s to get the easults and the FPS=
